# Project: Conway's Game Of Life 


## Overview

Conway’s Game of Life is a classic example of a **cellular automaton**. It consists of a grid of cells, each of which can be **alive** or **dead**. The grid evolves in discrete steps according to these simple rules, applied to each cell based on its eight neighbors:

1. Any live cell with fewer than two live neighbors dies (underpopulation).  
2. Any live cell with two or three live neighbors remains alive (survival).  
3. Any live cell with more than three live neighbors dies (overpopulation).  
4. Any dead cell with exactly three live neighbors becomes a live cell (reproduction).  

A number of animated gifs and visuals can be seen on the [Wiki page](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life).
## Learning Objectives

- Practice building up a project incrementally, starting with a straightforward solution and progressively introducing optimizations.  
- Practice applying optimization strategies, including selecting efficient data structures, minimizing unnecessary allocations, and reducing loop overhead.  
- Practice designing and running targeted tests to verify correctness at each development stage.  
- Practice measuring and analyzing performance using benchmarking and profiling tools.  

## Your Path Forward

Choose the path (or combination of paths) that best matches your interests, and document your experiments as you go.  

1. **Build the Naïve Solution**  
   Implement the simple, unoptimized Game of Life from scratch by following the four substeps (initialization, neighbor counting, state update, and visualization). If you get stuck or want a reference, check out the “Naïve Implementation: Four Substeps” code below.

2. **Profile the Baseline**  
   Take your solution o the naïve implementation solution below and use tools like `@time`, `@btime` (BenchmarkTools), and `Profile.@profile` to identify time consuming computation.

   
4. **Apply Optimizations**  
   Starting from the baseline, introduce performance improvements, potentially considering:
   - Swap `Matrix{Bool}` for `BitMatrix`  
   - Preallocate and reuse output buffers  
   - Add `@inbounds` and `@simd` annotations  
   - Ensure type-stable functions and eliminate globals  
   Measure the impact of each change and compare results to your profiled data.

5. **Follow Your Own Curiosity**  
   If neither profiling nor low-level tweaking appeals, explore something else that interests you:  
   - Build richer or interactive visualizations (e.g., with `Makie` or `Plotly`)  
   - Extend to custom rule sets or sparse/infinite grids  
   - Integrate a GUI
   - Anything else that fits your learning goals!

## Naïve Implementation: Four Substeps

Below is a simple, unoptimized version. You will use this as a baseline and then optimize on your own.

### Substep 1: Initialize the Grid
**Goal:** Initialize a 100×100 boolean grid with random `true`/`false` values to represent the starting state of the Game of Life.  
```{admonition} Substep 1 Solution
:class: dropdown
```Julia
using Random

const N = 100  # grid size
# Create a random N×N Matrix of Bool
grid = Matrix{Bool}(rand(Bool, N, N))
```
```

### Substep 2: Neighbor Counting
**Goal:** Count the number of live neighbors surrounding cell `(i, j)` in a finite `Matrix{Bool}` grid.  
```{admonition} Substep 2 Solution
:class: dropdown
```Julia
function count_neighbors_naive(grid::Matrix{Bool}, i::Int, j::Int)::Int
    cnt = 0
    for di in -1:1, dj in -1:1
        if di != 0 || dj != 0
            i2, j2 = i + di, j + dj
            if 1 ≤ i2 ≤ size(grid,1) && 1 ≤ j2 ≤ size(grid,2)
                cnt += grid[i2, j2]
            end
        end
    end
    return cnt
end
```
```

### Substep 3: Compute Next State Naïvely
**Goal:** Compute and return the next generation of the Game of Life grid by applying Conway’s rules, using a fresh `Matrix{Bool}` for the updated state.  
```{admonition} Substep 3 Solution
:class: dropdown
```Julia
function next_state_naive(grid::Matrix{Bool})::Matrix{Bool}
    N1, N2 = size(grid)
    newgrid = falses(N1, N2)
    for i in 1:N1, j in 1:N2
        cnt = count_neighbors_naive(grid, i, j)
        newgrid[i,j] = (grid[i,j] && (cnt == 2 || cnt == 3)) ||
                       (!grid[i,j] && cnt == 3)
    end
    return newgrid
end
```
```

### Substep 4: Run Simlation and Visualise
**Goal:** Animate the evolution of the Game of Life grid over a given number of steps and save the result as a GIF using Plots.  

```{admonition} Substep 4 Solution
:class: dropdown
```Julia
using Plots

function run_naive_simulation(grid::Matrix{Bool}, steps::Int)
    anim = @animate for _ in 1:steps
        grid = next_state_naive(grid)
        heatmap(
            grid;
            color = :grays, 
            axis = false, 
            legend = false,
            framestyle = :none,
            aspect_ratio = 1
        )
    end
    gif(anim, "life_naive.gif"; fps = 10)
    println("Saved naive animation to life_naive.gif")
end

# Example usage:
run_naive_simulation(grid, 100)
```
```

In [ ]:
To view the live implementation along with an animated GIF, please proceed to the next page of this website: